In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [19]:
data = pd.read_csv('/Users/tarlanjabiyev/Desktop/Sprint 20/b/moneyball.csv')
data.head()

,team,league,year,rs,ra,w,obp,slg,ba,playoffs,rankseason,rankplayoffs,g,oobp,oslg
0,ARI,NL,2012,734,688,81,0.328,0.418,0.259,0,NaN,NaN,162,0.317,0.415
1,ATL,NL,2012,700,600,94,0.320,0.389,0.247,1,4.0,5.0,162,0.306,0.378
2,BAL,AL,2012,712,705,93,0.311,0.417,0.247,1,5.0,4.0,162,0.315,0.403
3,BOS,AL,2012,734,806,69,0.315,0.415,0.260,0,NaN,NaN,162,0.331,0.428
4,CHC,NL,2012,613,759,61,0.302,0.378,0.240,0,NaN,NaN,162,0.335,0.424


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   team          1232 non-null   object 
 1   league        1232 non-null   object 
 2   year          1232 non-null   int64  
 3   rs            1232 non-null   int64  
 4   ra            1232 non-null   int64  
 5   w             1232 non-null   int64  
 6   obp           1232 non-null   float64
 7   slg           1232 non-null   float64
 8   ba            1232 non-null   float64
 9   playoffs      1232 non-null   int64  
 10  rankseason    244 non-null    float64
 11  rankplayoffs  244 non-null    float64
 12  g             1232 non-null   int64  
 13  oobp          420 non-null    float64
 14  oslg          420 non-null    float64
dtypes: float64(7), int64(6), object(2)
memory usage: 144.5+ KB


In [25]:
data.isna().sum()

team              0
league            0
year              0
rs                0
ra                0
w                 0
obp               0
slg               0
ba                0
playoffs          0
rankseason      988
rankplayoffs    988
g                 0
oobp            812
oslg            812
dtype: int64

## H2O

In [27]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_301"; Java(TM) SE Runtime Environment (build 1.8.0_301-b09); Java HotSpot(TM) 64-Bit Server VM (build 25.301-b09, mixed mode)
  Starting server from /opt/anaconda3/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/w5/0qbzsvvx23g_63j6m77_09nm0000gn/T/tmp5yd5d47v
  JVM stdout: /var/folders/w5/0qbzsvvx23g_63j6m77_09nm0000gn/T/tmp5yd5d47v/h2o_tarlanjabiyev_started_from_python.out
  JVM stderr: /var/folders/w5/0qbzsvvx23g_63j6m77_09nm0000gn/T/tmp5yd5d47v/h2o_tarlanjabiyev_started_from_python.err
  Server is running at http://127.0.0.1:54325
 successful.o H2O server at http://127.0.0.1:54325 ...
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Baku
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,5 months and 11 days
H2O_cluster_name:,H2O_from_python_tarlanjabiyev_zroj5i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.770 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [31]:
h2o_data = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [37]:
type(h2o_data)

h2o.frame.H2OFrame

## Datanın bölünməsi

In [42]:
train, valid, test = h2o_data.split_frame(ratios=[0.8, 0.1], seed=123)

In [78]:
target = 'rs'
features = data.drop(columns=[target]).columns.to_list()

['team',
 'league',
 'year',
 'ra',
 'w',
 'obp',
 'slg',
 'ba',
 'playoffs',
 'rankseason',
 'rankplayoffs',
 'g',
 'oobp',
 'oslg']

## AutoML

In [87]:
model = H2OAutoML(
    stopping_metric = "mae",
    nfolds = 10,
    seed = 123,
    max_runtime_secs = 360
)

model.train(
    x = features,
    y = target,
    training_frame = train,
    validation_frame = valid,
    leaderboard_frame = test
)

AutoML progress: |
15:01:11.99: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,10


In [89]:
model.leaderboard.as_data_frame()

/opt/anaconda3/lib/python3.12/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,model_id,rmse,mse,mae,rmsle,mean_residual_deviance
0,StackedEnsemble_BestOfFamily_4_AutoML_2_202509...,23.627407,558.254382,19.466741,0.033214,558.254382
1,StackedEnsemble_AllModels_3_AutoML_2_20250908_...,23.867942,569.678663,19.919099,0.033705,569.678663
2,DeepLearning_grid_1_AutoML_2_20250908_150111_m...,24.155672,583.496488,19.509510,0.033799,583.496488
3,DeepLearning_grid_1_AutoML_2_20250908_150111_m...,24.546886,602.549629,20.151581,0.034125,602.549629
4,XGBoost_3_AutoML_2_20250908_150111,25.583091,654.494550,20.208272,0.036499,654.494550
5,XGBoost_grid_1_AutoML_2_20250908_150111_model_11,25.587783,654.734653,20.351232,0.037103,654.734653
6,StackedEnsemble_AllModels_2_AutoML_2_20250908_...,25.846583,668.045853,21.111311,0.036480,668.045853
7,StackedEnsemble_AllModels_1_AutoML_2_20250908_...,25.879329,669.739653,21.150989,0.036942,669.739653
8,StackedEnsemble_BestOfFamily_1_AutoML_2_202509...,25.905648,671.102604,21.608587,0.036763,671.102604
9,StackedEnsemble_BestOfFamily_3_AutoML_2_202509...,26.094050,680.899469,21.171443,0.036597,680.899469


In [91]:
best_model = model.leader
best_model

key,value
Stacking strategy,cross_validation
Number of base models (used / total),5/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DeepLearning base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,10


## Reqressiya Modelinin Perormansının Qiymətləndirilməsi

In [94]:
best_model.model_performance(train=True).r2()

0.9723894444090272

In [96]:
best_model.model_performance(valid=True).r2()

0.9555480565537825

In [98]:
best_model.model_performance(xval=True).r2()

0.943178382657837

In [100]:
best_model.model_performance(test).r2()

0.9260618774153606

## Proqnozlar

In [103]:
y_pred = best_model.predict(test).as_data_frame()
y_pred

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


/opt/anaconda3/lib/python3.12/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,predict
0,778.916340
1,767.746874
2,628.623021
3,684.740267
4,820.909039
...,...
109,662.333879
110,595.021668
111,478.700216
112,592.073831


In [107]:
pred = y_pred.predict
pred

0      778.916340
1      767.746874
2      628.623021
3      684.740267
4      820.909039
          ...    
109    662.333879
110    595.021668
111    478.700216
112    592.073831
113    695.558098
Name: predict, Length: 114, dtype: float64